In [1]:
import numpy as np
import pandas as pd
import time
from dobbel import dobbellogger
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2] * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1 / 2
    w = np.cos(absq)
    sinabs = np.sin(absq)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])


def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

def quaternion_conjugate(q):
    w, x, y, z = q
    return np.array([w, -x, -y, -z])


# Actual Rotation Fuction - Input = Acceleration vector = 1x3 , rotation_quaternion = 1*4
def rotate_vector(vector, rotation_quaternion):
    # Convert the vector to a quaternion
    vector_quaternion = np.concatenate(([0], vector))

    # Calculate the rotated quaternion
    rotated_quaternion = quat_mul(rotation_quaternion,
                                  quat_mul(vector_quaternion, quaternion_conjugate(rotation_quaternion)))

    # Extract the rotated vector from the quaternion
    rotated_vector = rotated_quaternion[1:]

    return np.array(rotated_vector)

def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [3]:
angle = np.deg2rad(-39)
rotate_y = np.array([[np.cos(angle),0,np.sin(angle)], [0,1,0],[-np.sin(angle),0,np.cos(angle)]])
trace = np.trace(rotate_y)
q_0 = np.sqrt(1+trace)/2
q_2 = 1/(4*q_0) * (np.sin(angle)+np.sin(angle))
q_rotate = np.array([q_0,0,q_2,0])
print(q_rotate)

[ 0.94264149  0.         -0.33380686  0.        ]


In [32]:
x_means_list = []
y_means_list = []
z_means_list = []
x_gyro_list = []
y_gyro_list = []
z_gyro_list = []
x_gyro_std_list = []
y_gyro_std_list = []
z_gyro_std_list = []
# x_mag_list = []
# y_mag_list = []
# z_mag_list = []

#rotated accelerometer lists
xr_acc_list = []
yr_acc_list = []
zr_acc_list = []
xr_std_list = []
yr_std_list = []
zr_std_list = []

for i in range(3):
    i=i+1
    print("Leg de dobbelsteen met nummer ", i, " boven")
    dob = dobbellogger()
    dob.log(0.5, 200, 8, 125)
    dob.download()
    data = dob.datadf
    data = remove_nan(data)

    x_acc = data['x_acc']
    y_acc = data['y_acc']
    z_acc = data['z_acc']

    # Calculate the  mean
    x_mean = np.mean(x_acc)
    y_mean = np.mean(y_acc)
    z_mean = np.mean(z_acc)
    
    # Append the mean to the list
    x_means_list.append(x_mean)
    y_means_list.append(y_mean)
    z_means_list.append(z_mean)
    
    # Create a DataFrame from the list of means
    means_df = pd.DataFrame({'x_mean': x_means_list, 'y_mean': y_means_list, 'z_mean': z_means_list})
    means = np.array([x_mean,y_mean,z_mean])
  
    #calculate the std
    x_std = np.std(x_acc)
    y_std = np.std(y_acc)
    z_std = np.std(z_acc)
    # stds_df = pd.DataFrame({'x_std': x_std_list, 'y_std': y_std_list, 'z_std': z_std_list})
    stds = np.array([x_std, y_std, z_std])

    #rotate the means to the initial triad
    init = rotate_vector(means, q_rotate)
    std_init = rotate_vector(stds, q_rotate)
    
    #append the rotated means to the list
    xr_acc_list.append(init[0])
    yr_acc_list.append(init[1])
    zr_acc_list.append(init[2])

    xr_std_list.append(std_init[0])
    yr_std_list.append(std_init[1])
    zr_std_list.append(std_init[2])
    
    #Create a DataFrame from the list of rotated means
    init_df = pd.DataFrame({"xr_acc": xr_acc_list, "yr_acc": yr_acc_list, "zr_acc": zr_acc_list})
    init_std_df = pd.DataFrame({"xr_std": xr_std_list, "yr_std": yr_std_list, "zr_std": zr_std_list})
    
    # Calculate the  mean
    x_gyro = np.mean(data['x_gyro'])
    y_gyro = np.mean(data['y_gyro'])
    z_gyro = np.mean(data['z_gyro'])
    
    #calculate the std
    x_gyro_std = np.std(data['x_gyro'])
    y_gyro_std = np.std(data['y_gyro'])
    z_gyro_std = np.std(data['z_gyro'])

    # Append the mean to the list
    x_gyro_list.append(x_gyro)
    y_gyro_list.append(y_gyro)
    z_gyro_list.append(z_gyro)

    x_gyro_std_list.append(x_gyro_std)
    y_gyro_std_list.append(y_gyro_std)
    z_gyro_std_list.append(z_gyro_std)

    # Create a DataFrame from the list of means
    gyro_df = pd.DataFrame({'x_gyro': x_gyro_list, 'y_gyro': y_gyro_list, 'z_gyro': z_gyro_list})
    gyro_std_df = pd.DataFrame({'x_gyro_std': x_gyro_std_list, 'y_gyro_std': y_gyro_std_list, 'z_gyro_std': z_gyro_std_list})

    #
    # x_mag = np.mean(data['x_mag'])
    # y_mag = np.mean(data['y_mag'])
    # z_mag = np.mean(data['z_mag'])
    
    # x_mag_list.append(x_mag)
    # y_mag_list.append(y_mag)
    # z_mag_list.append(z_mag)
    # 
    # mag_df = pd.DataFrame({'x_mag': x_mag_list, 'y_mag': y_mag_list, 'z_mag': z_mag_list})
    

print(means_df)
print(init_df)
print(gyro_df)
print(gyro_std_df)
print(init_std_df)
# print(mag_df)

Leg de dobbelsteen met nummer  1  boven
Logging data for 0.5s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  2  boven
Logging data for 0.5s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  3  boven
Logging data for 0.5s
Done! The data is located in self.datadf
     x_mean    y_mean    z_mean
0 -0.648555 -0.040088 -0.826647
1 -0.037405  0.983096 -0.026762
2  0.746616 -0.017889 -0.656170
     xr_acc    yr_acc    zr_acc
0  0.016204 -0.040088 -1.050574
1 -0.012227  0.983096 -0.044338
2  0.993171 -0.017889 -0.040079
     x_gyro    y_gyro    z_gyro
0 -0.185389  0.439104  0.025100
1 -0.226630  0.455859 -0.004417
2 -0.202218  0.457571  0.019405
   x_gyro_std  y_gyro_std  z_gyro_std
0    0.081106    0.070282    0.113690
1    0.087918    0.062620    0.092937
2    0.078456    0.052383    0.059175
     xr_std    yr_std    zr_std
0  0.001172  0.005151  0.004525
1  0.000683  0.001804  0.003200
2 -0.000141  0.001404  0.002614


In [18]:
#Calibration algorithm
x_gyro_bias = np.mean(gyro_df['x_gyro'])
y_gyro_bias = np.mean(gyro_df['y_gyro'])
z_gyro_bias = np.mean(gyro_df['z_gyro'])
gyro_bias = np.array([x_gyro_bias,y_gyro_bias,z_gyro_bias])
print("gyro bias =", gyro_bias)

x_gyro_std = np.mean(gyro_std_df['x_gyro_std'])
y_gyro_std = np.mean(gyro_std_df['y_gyro_std'])
z_gyro_std = np.mean(gyro_std_df['z_gyro_std'])
gyro_std = np.array([x_gyro_std, y_gyro_std, z_gyro_bias])
print("gyro std =", gyro_std)

gyro bias = [-0.14534332  0.46132693  0.01679937]
gyro std = [0.07661315 0.06601597 0.01679937]


In [36]:
#remove the gravity for the bias
xr_bias_acc = 1-np.abs(xr_acc_list[2])
yr_bias_acc = 1-np.abs(yr_acc_list[1])
zr_bias_acc = 1-np.abs(zr_acc_list[0])
bias = np.array([xr_bias_acc, yr_bias_acc, zr_bias_acc])
print("accelerometer bias = ", bias)

xr_std = xr_std_list[2]
yr_std = yr_std_list[1]
zr_std = zr_std_list[0]
acc_std = np.array([xr_std, yr_std, zr_std])
print("accelerometer std = ", acc_std)

accelerometer bias =  [ 0.00682937  0.0169035  -0.05057431]
accelerometer std =  [-0.00014147  0.00180437  0.00452501]


In [37]:
print(std_init_df)

     xr_std    yr_std    zr_std
0  0.000613  0.001716  0.003413
